# Análise da Emissão de Gases do Efeito Estufa em Brasília

- Alunos: Gabriel da Silva Loures e Kauã Ferdandes da Fonseca de Oliveira.

## Objetivo
Este notebook tem como objetivo analisar os dados de emissões de gases de efeito estufa (GEE) em Brasília(DF), com base no arquivo "gases.csv". A análise será dividida em quatro etapas principais:

1.  **Carregamento e Limpeza dos Dados:** Preparação do dataset para a análise.
2.  **Análise Exploratória de Dados (AED):** Extração de informações e respostas a perguntas específicas.
3.  **Visualização de Dados:** Criação de gráficos para ilustrar as descobertas.
4.  **Conclusões:** Resumo dos principais insights obtidos.

In [6]:
# Importação das bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração do estilo dos gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Carregamento e Limpeza dos Dados

A primeira etapa é carregar os dados e prepará-los para a análise. Isso envolve corrigir o formato da tabela, verificar os tipos de dados e tratar valores ausentes.

In [13]:
# 1.1. Carregar o arquivo gases.csv em um DataFrame
try:
    df_raw = pd.read_csv('gases.csv')
    print("Arquivo 'gases.csv' carregado com sucesso.")
except FileNotFoundError:
    print("Erro: Arquivo 'gases.csv' não encontrado. Verifique se o arquivo está na mesma pasta.")
    df_raw = pd.DataFrame() # Cria um DataFrame vazio para evitar erros posteriores

if not df_raw.empty:
    # 1.2. Transformação do formato da Tabela (de "larga" para "longa")
    # O formato original possui anos como colunas, o que não é ideal para análise.
    # Vamos "derreter" (melt) o DataFrame para ter uma coluna 'Ano' e uma coluna 'Emissao'.

    # Identifica as colunas que são de identificação e as que são de valores (anos)
    id_cols = list(df_raw.columns[:11])
    year_cols = list(df_raw.columns[11:])

    df = df_raw.melt(
        id_vars=id_cols,
        value_vars=year_cols,
        var_name='Ano',
        value_name='Emissao'
    )

    print("\nFormato da tabela transformado com sucesso.")

    # 1.3. Verificação e correção dos tipos de dados
    print("\nVerificando e corrigindo os tipos de dados...")
    df['Ano'] = pd.to_numeric(df['Ano']) # Converte a coluna 'Ano' para número
    df['Emissao'] = pd.to_numeric(df['Emissao'], errors='coerce') # Converte 'Emissao' para número

    # 1.4. Identificação e tratamento de valores ausentes (NaN)
    # Vamos assumir que valores ausentes (NaN) significam zero emissões para aquele registro.
    valores_ausentes_antes = df['Emissao'].isnull().sum()
    print(f"Valores ausentes encontrados na coluna 'Emissao': {valores_ausentes_antes}")

    df.fillna({'Emissao': 0}, inplace=True)
    print("Valores ausentes preenchidos com 0.")

    # Exibe as informações e as primeiras linhas do DataFrame limpo e pronto para análise
    print("\n--- Informações do DataFrame Tratado ---")
    df.info()

    print("\n--- 5 Primeiras Linhas do DataFrame Tratado ---")
    display(df.head(500))

Arquivo 'gases.csv' carregado com sucesso.

Formato da tabela transformado com sucesso.

Verificando e corrigindo os tipos de dados...
Valores ausentes encontrados na coluna 'Emissao': 8
Valores ausentes preenchidos com 0.

--- Informações do DataFrame Tratado ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83268 entries, 0 to 83267
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Setor de emissão        83268 non-null  object 
 1   Categoria emissora      83268 non-null  object 
 2   Sub-categoria emissora  83268 non-null  object 
 3   Produto ou sistema      83268 non-null  object 
 4   Detalhamento            83268 non-null  object 
 5   Recorte                 77328 non-null  object 
 6   Atividade geral         83268 non-null  object 
 7   Bioma                   82782 non-null  object 
 8   Emissão/Remoção/Bunker  83268 non-null  object 
 9   Gás                     83268 non-null

,Setor de emissão,Categoria emissora,Sub-categoria emissora,Produto ou sistema,Detalhamento,Recorte,Atividade geral,Bioma,Emissão/Remoção/Bunker,Gás,Cidade,Ano,Emissao
0,Agropecuária,Cultivo de arroz,Cultivo em sistema irrigado inundado,Arroz,Vegetal,Diretas,Agricultura,Cerrado,Emissão,CH4 (t),Brasília (DF),1970,2.937752
1,Agropecuária,Fermentação entérica,Processo de digestão de animais ruminantes,Asinino,Animal,Diretas,Pecuária,Cerrado,Emissão,CH4 (t),Brasília (DF),1970,0.630000
2,Agropecuária,Fermentação entérica,Processo de digestão de animais ruminantes,Bubalino,Animal,Diretas,Pecuária,Cerrado,Emissão,CH4 (t),Brasília (DF),1970,0.000000
3,Agropecuária,Fermentação entérica,Processo de digestão de animais ruminantes,Caprino,Animal,Diretas,Pecuária,Cerrado,Emissão,CH4 (t),Brasília (DF),1970,2.515000
4,Agropecuária,Fermentação entérica,Processo de digestão de animais ruminantes,Equino,Animal,Diretas,Pecuária,Cerrado,Emissão,CH4 (t),Brasília (DF),1970,55.854000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Energia,Industrial,Mineração e pelotização,Diesel de petróleo,Centrais elétricas autoprodutoras,Emissões pela queima de combustíveis,Geração de eletricidade,Cerrado,Emissão,CO2 (t),Brasília (DF),1970,0.000000
496,Energia,Industrial,Mineração e pelotização,Diesel de petróleo,Centrais elétricas autoprodutoras,Emissões pela queima de combustíveis,Geração de eletricidade,Cerrado,Emissão,CO2 (t),NA (DF),1970,0.000000
497,Energia,Industrial,Mineração e pelotização,Diesel de petróleo,Centrais elétricas autoprodutoras,Emissões pela queima de combustíveis,Geração de eletricidade,Cerrado,Emissão,CO (t),Brasília (DF),1970,0.000000
498,Energia,Industrial,Mineração e pelotização,Diesel de petróleo,Centrais elétricas autoprodutoras,Emissões pela queima de combustíveis,Geração de eletricidade,Cerrado,Emissão,CO (t),NA (DF),1970,0.000000
